# План на сегодня

1. классовый метод, декоратор @classmethod
2. статический метод, декоратор @staticmethod
3. свойство, декоратор @property
7. протокол итерации
8. `Callable` и некоторые встроенные функции
1. Наследование

## Декораторы classmethod, staticmethod, property

По умолчанию при объявлении метода в теле класса его первый параметр используется как ссылка на экземпляр, но иногда нам нужны методы, которые можно было бы вызвать без экземпляра класса. Такие методы бывают двух типов: [классовые](https://docs.python.org/3/library/functions.html#classmethod) и [статические](https://docs.python.org/3/library/functions.html#staticmethod). 

Изменить поведение "первый аргумент - ссылка на экземпляр" и превратить методы в статические или классовые можно с помощью специальных **встроенных** декораторов `staticmethod` и `classmethod`. Мы еще не изучали принципы работы декораторов, но пока что нам надо лишь использовать их и посмотреть на результат. Для того, чтобы **задекорировать** метод, нужно перед его объявлением написать `@name_of_decorator`:

In [ ]:
class MyClass:
    
    def method(self):
        print('Обычные методы можно назвать методами уровня экземпляра класса.')
    
    @staticmethod
    def static_sum(arg1, arg2):  # нет self
        print('статический метод умеет работать только с переданными ему аргументами\n'
              'и не имеет доступа к классу или его экземплярам')
        print(arg1 + arg2) 
        
    @classmethod
    def clsmethod(cls):  # первый аргумент теперь ссылка на сам класс
        print(cls.__name__)
        print('Метод уровня класса имеет доступ к классу.\n' 
              'Часто используется для создания новых экземпляров класса')

Классовые методы можно назвать методами уровня класса. Их особенность в том, что первым аргументом они принимают ссылку не на экземпляр `self`, а на сам класс - `cls` (также имя по соглашению).

Особенность статических в том, что им не нужна ссылка ни на экземпляр, ни на класс.

In [ ]:
instance = MyClass()
instance.method()

In [ ]:
MyClass.static_sum(1, 41)

In [ ]:
MyClass.clsmethod()

Все эти методы все равно можно вызывать и от экземпляров:

In [ ]:
instance.static_sum(1, 41)
instance.clsmethod()

Далее разберемся с декоратором `property`.

### property

https://docs.python.org/3/library/functions.html#property

Выше мы обсудили, что для получения значений атрибутов принято писать getter'ы, а для изменения значений - setter'ы. Добавлю, что внутренняя логика этих методов может быть сильно шире, например, можно написать проверку типа пришедшего объекта или допустимый диапазон какого-нибудь значения. Для простоты примеров мы это не используем.

Вернемся к нашему классу `Figure` и перепишем его, создав getter для атрибута `_color` с помощью декоратора `property`:

In [ ]:
class Figure:
    
    def __init__(self, color):
        self._color = color
    
    @property                    # устанавливает getter
    def color(self):
        print('вызван геттер')
        return self._color

In [ ]:
square = Figure('red')
square.color

Обратите внимание, что мы получили значение цвета, обратившись по имени `color`, а не `_color`. Но это не все, ведь теперь изменить значение атрибута у нас не получится:

In [ ]:
square.color = 'green'

Чтобы можно было менять цвет, нужно установить и setter следующим кодом:

In [ ]:
class Figure:
    
    def __init__(self, color):
        self._color = color
    
    @property                    # устанавливает getter
    def color(self):
        print('вызван геттер')
        return self._color
    
    @color.setter                # устанавливает setter
    def color(self, new_color):
        if isinstance(new_color, str):
            print('вызван сеттер')
            self._color = new_color

In [ ]:
square = Figure('red')

In [ ]:
square.color

In [ ]:
square.color = 'green'
square.color

Значение цвета хранится в приватной переменной `_color`, а новое свойство `color` "было создано" для нас декоратором `property`.

**Note**: property (переводится как "свойство") — это способ доступа к внутреннему состоянию объекта. Обращение к свойству выглядит так же, как и обращение к атрибуту, но, в действительности, реализовано через вызов getter'а. Изменение значения свойства вызывает setter. Вообще еще можно установить так называемый deleter, который позволяет удалить атрибут, если надо:

In [ ]:
class Figure:
    
    def __init__(self, color):
        self.color = color       # обратите внимание на изменение имени атрибута!
    
    @property                    # устанавливает getter
    def color(self):
        print('вызван getter')
        return self._color
    
    @color.setter                # устанавливает setter
    def color(self, new_color):
        print('вызван setter')
        self._color = new_color
    
    @color.deleter               # устанавливает deleter
    def color(self):
        print('вызван deleter')
        del self._color

In [ ]:
square = Figure('red')

При создании квадрата мы видим, что вызвался `setter`, так как раньше в `__init__` мы создавали атрибут `self._color`, а теперь там находится `self.color`, но ведь теперь `color` - имя свойства! Поэтому инструкция `self.color = color` в методе `__init__` вызывает setter.

In [ ]:
square.color

In [ ]:
del square.color

Необязательно определять все три метода для каждого свойства. Вы можете определить свойства, доступные только для чтения, установив только getter. Если значение атрибута будет меняться, то можно определить и setter. Ну а для удаления - deleter.

## Где-то мы это уже видели: протокол итератора

Вспомним основные тезисы:

1. Чтобы контейнер был итерабельным, у него должен быть определен метод `__iter__`, возвращающий итератор. Недостаток итераторов в том, что они хранят сразу все значения, то есть могут занимать много оперативной памяти.

2. У итератора должны быть определены методы `__next__` (возвращает следующий элемент из контейнера) и `__iter__` (обычно просто возвращает самого себя - `self`)

3. Если итератор исчерпан (отдал все элементы), то метод `__next__` **всегда** должен `StopIteration`

4. Генератор - специальный вид итератора. Они не хранят все объекты в оперативной памяти, а создают их при обращении - такой принцип работы называется "ленивые вычисления". При создании генератора методы `__next__` и `__iter__` определяются автоматически, то есть можно сказать, что интерфейс итератора реализуется неявно.

5. Генераторы создаются либо через определение функции, которая возвращает объект генератора, если в ней присутствует `yield statement`, либо с помощью `generator expression`, также возвращающее объект генератора.

В качестве примера итератора давайте напишем ненастоящий range, который будет выполнять проход с единичным шагом от `0` до заданного `stop` включительно:

In [ ]:
class MySimpleRange:
    
    def __init__(self, stop):
        self.start = 0
        self.stop = stop

    def __iter__(self):
        return self

    def __next__(self):
        if self.start <= self.stop:            
            start = self.start
            self.start += 1
            return start
        else:
            raise StopIteration

In [ ]:
iterator = MySimpleRange(3)
print(iterator.start, iterator.stop)

print(iterator.__next__())
print(iterator.__next__())
print(iterator.__next__())
print(iterator.__next__())

При следующем вызове `next` мы бы получили `StopIteration`, но поскольку `for statement` самостоятельно обрабатывает это исключение, то обычно нам отдельно его обрабатывать не надо:

In [ ]:
for value in MySimpleRange(3):
    print(value)

У итераторов есть некоторые ограничения. Например, нельзя узнать его длину, а после прохода по нему сделать это еще раз не получится:

In [ ]:
for value in iterator:
    print(value)
else:
    print('Итератор опустел. Для нового прохода его надо пересоздать')

Самостоятельно убедитесь, что с итераторами:
- нельзя достать элемент по индексу
- нельзя взять срез
- но можно взять срез с помощью `itertools.islice`
- их можно распаковывать `*`
- и использовать везде, где нужны итераторы

In [ ]:
print(*MySimpleRange(2))

Поскольку теперь нам больше известно об атрибутах объектов, то посмотрим в атрибуты объекта типа `range`:

In [ ]:
ran = range(3)

print(type(ran))

ran.start, ran.stop, ran.step

И создадим его аналог*, но в виде генератора:

In [ ]:
def my_range(start, stop=None, step=1):
    if stop is None:
        stop = start
        start = 0
    i = start
    while i < stop:
        yield i
        i += step

for value in my_range(2, 7, 2):
    print(value)

print()    
print(type(my_range))
print(type(my_range(3)))  # вызвали my_range

print()
# убедимся, что все методы итератора действительно есть
print(my_range(3).__iter__)
print(my_range(3).__iter__())
print(my_range(3).__next__)

## Встроенные функции

### `getattr`

```python
getattr(obj, name[, default]) -> value
```

Получает атрибут объекта: вызов `getattr(x, 'y')` эквивалентен `x.y`

In [ ]:
getattr(square, '_color')

In [ ]:
square._color

Можно задать дефолтное значение:

In [ ]:
getattr(square, 'this_attr_does_not_exist', 42)

In [ ]:
dict.get()

### `hasattr`

```python
hasattr(obj, name, /)
```

Возвращает `True`, если объект обладает атрибутом с заданным именем:

In [ ]:
hasattr(square, '_color')

In [ ]:
hasattr(square, 'this_attr_does_not_exist')

При этом проверка наличия атрибута происходит вызовом `getattr(obj, name)`:

In [ ]:
hasattr(square, 'color')

### `setattr`

```python
setattr(obj, name, value, /)
```

Присваивает атрибуту `name` данного объекта `obj` указанное значение `value`.

`setattr(obj, 'name', value)` эквивалентно `obj.name = value`

In [ ]:
setattr(square, 'this_attr_does_not_exist', 42)

In [ ]:
getattr(square, 'this_attr_does_not_exist')

In [ ]:
setattr(square, 'color', 42)

### `delattr`

```python
delattr(obj, name, /)
```

Удаляет атрибут `name` заданного объекта `obj`.

`delattr(obj, name)` эквивалентно `del obj.name`

In [ ]:
delattr(square, 'this_attr_does_not_exist')

### `callable`

```python
callable(obj, /) -> bool
```

Возвращает `True`, если объект является вызываемым:

In [ ]:
callable(len)

In [ ]:
callable(5)

Оператор вызова объекта - круглые скобки справа от выражения, возвращающего этот объект.

Все классы являются вызываемыми, как и функции, а, например, числа, списки и строки (экземпляры классов `int/float/complex`, `list`, `str`) вызвать не получится:

In [ ]:
[]()

Но можно ли сделать экземпляр класса вызываемым? Да, для этого надо переопределить специальный метод `__call__`:

In [ ]:
class Summator:
    
    def __init__(self, x=0):
        print('вызван инит')
        self.x = x

    def __call__(self, y):
        print('вызван метод __call__')
        return self.x + y

In [ ]:
sum10 = Summator(10)
sum42 = Summator(42)

In [ ]:
sum10.x

In [ ]:
sum10(1000)

In [ ]:
sum10(10)

In [ ]:
sum42(0)

## Наследование

**Наследование**: создание нового класса на основе уже существующего. При этом наследуются его атрибуты и методы.

- класс-потомок - child class - subclass
- родительский класс - base class

Прародителем всех классов в Python является `object`

In [ ]:
object

In [ ]:
class Animal:

    some_value = "animal"

    def __init__(self):
        print("i am an animal")

    def speak(self):
        raise NotImplementedError('i don\'t know how to speak')

In [ ]:
animal = Animal()

In [ ]:
animal.some_value

In [ ]:
animal.speak()

In [ ]:
class Cat(Animal):

    some_value = "cat"

    def __init__(self):
        super().__init__()
        print("i am a cat")

    def speak(self):
        print('meoooow')

In [ ]:
animal = Animal()
animal.some_value

In [ ]:
cat = Cat()
cat.some_value  # переопределено

In [ ]:
class Hedgehog(Animal):

    def __init__(self):
        super().__init__()
        print("i am a hedgehog")

In [ ]:
hedgehog = Hedgehog()
hedgehog.some_value  # не переопределено

In [ ]:
class Dog(Animal):

    some_value = "dog"

    def __init__(self):
        super().__init__()
        print("i am a dog")

In [ ]:
dog = Dog()
dog.some_value  # переопределено

In [ ]:
class CatDog(Cat, Dog):  # ромбовидное наследование возможно

    def __init__(self):
        super().__init__()
        print("i am a CatDog!")

In [ ]:
catdog = CatDog()
catdog.some_value

In [ ]:
catdog.speak()

In [ ]:
# ______Animal______
# ___/    |    \
# Cat   Dog   Hedgehog
#    \   /
#    CatDog     catdog.speak

In [ ]:
class CatDog(Dog, Cat):  # теперь наоборот, найдите два отличия!
    def __init__(self):
        super().__init__()
        print("i am a CatDog!")

catdog = CatDog()
catdog.some_value

In [ ]:
catdog.speak()

In [ ]:
cat.speak()  # переопределено
dog.speak()  # не переопределено

In [ ]:
def f():
    print('aaaaa')
    g()
    print('AAAAAA')


def g():
    print('bbbbbb')
    print('BBBBBBB')

f()

In [ ]:
class A:
    def method(self):
        print('I\'m "class A"')
        # super().method()
        print('I\'m here "class A"')


class B(A):
    def method(self):
        print('I\'m "class B"')
        super().method()
        print('I\'m here "class B"')

class C(A):
    def method(self):
        print('I\'m "class C"')
        super().method()
        print('I\'m here "class C"')

class D(B, C):
    def method(self):
        print('I\'m "class D"')
        super().method()
        print('I\'m here "class D"')

class E:
    def method(self):
        print('I\'m "class E"')
        super().method()
        print('I\'m here "class E"')

class F(E, D):
    def method(self):
        print('I\'m "class F"')
        super().method()
        print('I\'m here "class F"')


f = F()
f.method()

In [ ]:
print(F.mro())

In [ ]:
F.__mro__

In [ ]:
D.__mro__

In [ ]:
D.__mro__ = (D, C, B, A, object)

In [ ]:
print(isinstance(1, (int, str)))
print(isinstance(True, int))
print(isinstance(Animal, object))
print(isinstance(F, object))
print(isinstance(42, str))
print(issubclass(F, B))
print(issubclass(D, (E, C)))
print(isinstance(F, B))

In [ ]:
D.__bases__

## Документация:

- [Tutorial 9. Classes](https://docs.python.org/3/tutorial/classes.html)
- [Data Model 3.2.13.5 Static method objects](https://docs.python.org/3/reference/datamodel.html#static-method-objects)
- [Data Model 3.2.13.6 Class method objects](https://docs.python.org/3/reference/datamodel.html#class-method-objects)
- [Data Model 3.3.6. Emulating callable objects](https://docs.python.org/3/reference/datamodel.html#emulating-callable-objects)
- [HOWTO: Method Resolution Order](https://docs.python.org/3/howto/mro.html#python-2-3-mro)

https://en.wikipedia.org/wiki/C3_linearization